In [1]:
from lxml import etree
import json
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline  
import seaborn as sns
import regex as reg
import unicodedata

In [2]:
import numpy as np
import regex #

# Parsing the metadata:

This notebook contains a more annotated indepth explination on the code implememted in the parsing script:

The following function extracts the metadata provided in the xml files. We iteratively parse the file as it does not follow standard xml format perfectly, hence using a regular parser would lead to errors being thrown.  The reason the xml can not be parsed with a regular paser is that the following tag is not present: `xmlns:dc ="http://purl.org/dc/elements/1.1/"` . This is not an issue in the web version. 

The information we want to extract is in the sections of the qualifier, element and subject attributes.

In [3]:
def parse(file_path): 
    i = 0 
    tmp = ''
    key = ''
    xml2 = etree.iterparse(file_path, recover=True)
    data = []
    for action, elem in xml2:
        data.append((elem.attrib, elem.tag, elem.text))
    data_dict = {}
    
    
    for attrib, tag, text in data: 
        try : 
            tmp = key
            
            key = attrib.get('qualifier')
            element = attrib.get('element')
            
            #way to distinguish eliminate some nan!
            if key == 'none':
                key = element
            
            if key in data_dict.keys() : 
                i = i + 1 
                data_dict[key + str(i)] = text
            else : 
                i = 0 
                data_dict[key] = text
                
        except TypeError: 
            if 'subject' in tag:
                if 'subject' in data_dict.keys():
                    data_dict['subject'].append(text)
                else:
                    data_dict['subject'] = [text]
                    
        #parse irregular files
        if 'subject' not in data_dict.keys():
            f = open(file_path)
            text = f.read()
            data_dict['subject'] = reg.findall('&gt;([\w\-\ \;]*)&lt;', text)
            if len(data_dict['subject']) == 1:
                data_dict['subject'] = reg.split(';', data_dict['subject'][0])
            
    return data_dict

### Get all data 

We go over all the folders and parse all the xml files

In [4]:
import os
rootdir = '../papers-import/'

In [5]:
all_data = {}
i= 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        path = os.path.join(subdir, file)
        if 'dublin_core' in (path) :
            i += 1
            num_doc = subdir[len(rootdir):]
            all_data[num_doc] = parse(path)

In [6]:
print('got data from {} files'.format(i))

got data from 1125 files


In [7]:
#add all the xml data into one dataframe
df_data = pd.DataFrame(list(all_data.values()), index=all_data.keys())

Convert to datetime format:

In [8]:
def convert(arg): 
    """Convert string to date time format"""
    try : 
        arg = dateutil.parser.parse(arg)
    except TypeError: 
        arg = arg 
    return arg 

In [9]:
df_data

,citation,uri,None,publisher,accessioned,available,iso,title,type,author,...,author21,author19,author16,author17,author20,author22,author23,author26,author24,author25
2016/12,"Prinsen, F. R. (2016). Supporting Inquiry Lear...",info:doi/10.22318/icls2016.12,\n,Singapore: International Society of the Learni...,2017-03-21T12:05:42Z,2017-03-21T12:05:42Z,en,Supporting Inquiry Learning as a Practice: A P...,Book chapter,"Prinsen, Fleur R.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014/1547-1548,"Krishnan, G. & Sengupta, P. (2014). Learning I...",https://doi.dx.org/10.22318/icls2014.1547,\n,"Boulder, CO: International Society of the Lear...",2018-05-21T22:08:51Z,2018-05-21T22:08:51Z,en,Learning Integrated STEM Using Tangible Agent-...,Book chapter,"Krishnan, Gokul",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014/641-648,"Chen, Y. & Hand, B. (2014). The Roles of Teach...",https://doi.dx.org/10.22318/icls2014.641,\n,"Boulder, CO: International Society of the Lear...",2018-05-21T22:08:51Z,2018-05-21T22:08:51Z,en,The Roles of Teacher Questioning in Argument-b...,Book chapter,"Chen, Ying-Chih",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018/367,"Uttamchandani, S. & Peppler, K. A. (2018). Rec...",https://doi.dx.org/10.22318/cscl2018.1509,math,International Society of the Learning Science...,2018-11-04T23:35:55Z,2018-11-04T23:35:55Z,en,Recounting Counting: Situated Self-Appraisal i...,Book chapter,"Uttamchandani, Suraj",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015/137,"Kim, B. (2015). Playing with Gameful Activitie...",https://doi.dx.org/10.22318/cscl2015.137,gameful assessment,International Society of the Learning Science...,2017-06-19T14:51:54Z,2017-06-19T14:51:54Z,en,Playing with Gameful Activities and Assessment...,Book chapter,"Kim, Beaumie",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018/294,"Wallon, R. C. & Lindgren, R. (2018). Cueing Ge...",https://doi.dx.org/10.22318/cscl2018.1363,embodied learning,International Society of the Learning Science...,2018-11-04T23:35:54Z,2018-11-04T23:35:54Z,en,Cueing Gestures in a Seasons Simulation: Outco...,Book chapter,"Wallon, Robert C",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014/1497-1498,"Dasgupta, C. & Moher, T. (2014). Using Deficie...",https://doi.dx.org/10.22318/icls2014.1497,\n,"Boulder, CO: International Society of the Lear...",2018-05-21T22:08:51Z,2018-05-21T22:08:51Z,en,Using Deficient Models as Scaffolds for Learni...,Book chapter,"Dasgupta, Chandan",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016/35,"Tay, W. Y., Chai, B. Y., Khamis, N. A., Tan, S...",info:doi/10.22318/icls2016.35,\n,Singapore: International Society of the Learni...,2017-03-21T12:05:42Z,2017-03-21T12:05:42Z,en,Towards a Framework of Pedagogical Paradoxes: ...,Book chapter,"Tay, Wan Ying",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018/402,"Duvall, M., Lee, F. J., & Smith, B. (2018). Sk...",https://doi.dx.org/10.22318/cscl2018.1579,professional development,International Society of the Learning Science...,2018-11-04T23:35:55Z,2018-11-04T23:35:55Z,en,Skyscraper Games: Designing Professional Devel...,Book chapter,"Duvall, Matthew",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014/543-550,"Tierney, G. (2014). Defining Success in an Alt...",https://doi.dx.org/10.22318/icls2014.543,\n,"Boulder, CO: International Society of the Lear...",2018-05-21T22:08:51Z,2018-05-21T22:08:51Z,en,Defining Success in an Alternative High School...,Book chapter,"Tierney, Gavin",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
import dateutil.parser
df_data['available'] = df_data['available'].apply(lambda x : convert(x))
df_data['accessioned'] = df_data['accessioned'].apply(lambda x : convert(x))

List of columns in the dataframe. We have one paper per row, with authors in the author columns:

In [11]:
df_data.columns

Index([   'citation',         'uri',          None,   'publisher',
       'accessioned',   'available',         'iso',       'title',
              'type',      'author',      'issued',     'subject',
          'abstract',     'author1',     'author3',     'author4',
           'author2',     'author5',     'author6',     'author7',
           'author8',     'author9',    'author10',    'author11',
          'author12',    'author14',    'author13',    'author15',
          'author18',    'author21',    'author19',    'author16',
          'author17',    'author20',    'author22',    'author23',
          'author26',    'author24',    'author25'],
      dtype='object')

# Getting all authors in to one dataframe:

Now we will change the above dataframe structure so that we only have one author column:

In [12]:
#go from multiple author columns to a single one
cleaning = df_data.reset_index().melt(
    id_vars=['index','subject', 'iso', 'uri','type','publisher','title', 
             'issued', 'accessioned', 'citation', 'available', 'abstract'])
cleaning = cleaning[cleaning.value.notna()]
cleaning = cleaning[cleaning.variable.notna()]

#remove garbage entries
cleaning = cleaning[cleaning.value.map(lambda x: len(x) > 2)]
#remove na
cleaning = cleaning[cleaning.value.notna()]

Associate an order to the authors: this will later be used to merge with the exracted emails:

In [13]:
import re
cleaning['author_order'] = cleaning.variable.map(
    lambda x: 0 if len(re.search('\d*$', x).group(0)) == 0 else int(re.search('\d*$', x).group(0)))

del cleaning['variable']

The dataframe currently looks like this:

In [14]:
cleaning.sort_values(by='index').head()

,index,subject,iso,uri,type,publisher,title,issued,accessioned,citation,available,abstract,value,author_order
5150,2014/1002-1006,[],en,https://doi.dx.org/10.22318/icls2014.1002,Book chapter,"Boulder, CO: International Society of the Lear...",Teaching Struggling Middle School Readers to C...,2014-06,2018-05-21 22:08:51+00:00,"Caccamise, D., Friend, A., Groneman, C., Littr...",2018-05-21 22:08:51+00:00,This project involves developing a reading com...,"Kintsch, Eileen",4
6275,2014/1002-1006,[],en,https://doi.dx.org/10.22318/icls2014.1002,Book chapter,"Boulder, CO: International Society of the Lear...",Teaching Struggling Middle School Readers to C...,2014-06,2018-05-21 22:08:51+00:00,"Caccamise, D., Friend, A., Groneman, C., Littr...",2018-05-21 22:08:51+00:00,This project involves developing a reading com...,"Groneman, Christine",2
1775,2014/1002-1006,[],en,https://doi.dx.org/10.22318/icls2014.1002,Book chapter,"Boulder, CO: International Society of the Lear...",Teaching Struggling Middle School Readers to C...,2014-06,2018-05-21 22:08:51+00:00,"Caccamise, D., Friend, A., Groneman, C., Littr...",2018-05-21 22:08:51+00:00,This project involves developing a reading com...,"Caccamise, Donna",0
2900,2014/1002-1006,[],en,https://doi.dx.org/10.22318/icls2014.1002,Book chapter,"Boulder, CO: International Society of the Lear...",Teaching Struggling Middle School Readers to C...,2014-06,2018-05-21 22:08:51+00:00,"Caccamise, D., Friend, A., Groneman, C., Littr...",2018-05-21 22:08:51+00:00,This project involves developing a reading com...,"Friend, Angela",1
4025,2014/1002-1006,[],en,https://doi.dx.org/10.22318/icls2014.1002,Book chapter,"Boulder, CO: International Society of the Lear...",Teaching Struggling Middle School Readers to C...,2014-06,2018-05-21 22:08:51+00:00,"Caccamise, D., Friend, A., Groneman, C., Littr...",2018-05-21 22:08:51+00:00,This project involves developing a reading com...,"Littrell-Baez, Megan",3


In [15]:
cleaning[cleaning.subject.map(len) == 0]['index'].unique()

array(['2014/1547-1548', '2014/641-648', '2014/1497-1498', '2014/543-550',
       '2014/1157-1161', '2014/871-878', '2014/1047-1051', '2014/681-688',
       '2014/800-807', '2014/262-269', '2014/1661-1662', '2014/1517-1518',
       '2014/126-133', '2014/1162-1165', '2014/294-300', '2014/70-77',
       '2014/1332-1341', '2014/713-720', '2014/1396-1405',
       '2014/1543-1544', '2014/721-728', '2014/1342-1351',
       '2014/1466-1473', '2014/1617-1618', '2014/1643-1644',
       '2014/110-117', '2014/1352-1361', '2014/1315-1322', '2018/480',
       '2014/1555-1556', '2014/1481-1482', '2014/1509-1510',
       '2014/1615-1616', '2014/887-894', '2014/1416-1425',
       '2014/1710-1710', '2014/1565-1566', '2014/394-401', '2014/863-870',
       '2014/535-542', '2014/831-838', '2014/1659-1660', '2014/1709-1709',
       '2018/516', '2014/1323-1331', '2014/1709-1708', '2014/911-918',
       '2014/1489-1490', '2014/957-961', '2017/116', '2014/1605-1606',
       '2014/847-854', '2014/1619-1620', '

In [16]:
len(cleaning[cleaning.subject.map(len) == 0]['index'].unique())

359

## Now parse citation to get the shortened name (which can be matched to refrences)

the `get_names` variable matches the general format of surnames in citations
the `section_before_year` identifies parts with contain the names in APA style citation, by looking for the section before the (year) which comes right after the names of the authors in this citation style

In [17]:
get_names = r'([\w\-\&]*[\,] [\p{L}\.\ ]+[\&\,]?)'
section_before_year = r'[\S\s]*\(\d{4}\)'

cleaning.reset_index(drop=True, inplace=True)

cleaning['shortend_names'] = cleaning.citation.map(lambda x: re.match(section_before_year, x, re.U).group(0)).map(
    lambda x: [x.replace(',', '').replace('&', '').rstrip() for x in regex.findall(get_names, x)])

cleaning['shortend_names'] = cleaning.apply(lambda x: x['shortend_names'][x['author_order']], axis=1)

Do some renaming of the columns to make them easier to understand:

In [18]:
cleaning.rename(columns={'index': 'file', 'value':'long_name'}, inplace=True)

In [19]:
#make the file name match the convention
cleaning['file'] = cleaning.file.map(lambda x: x.replace('/', '_'))

Extracting the identifier string, which can be used to match references in paper to papers in the dataset:

In [20]:
def get_authors_month(sentence, debug = False):
    """Gets name of authors + reference to time of publication"""
    sentence = unicodedata.normalize('NFC', sentence)
    #regex to match the pattern presented, people are misspelling prone, hence the long regex
    regex = r'[\p{L}\,\ \.\:\;\/\&\-\'\`\(\)\’\–\¨\…\‐\*\´\＆\\]*\([\,\ \p{L}\d\-]*(18|19|20)\d{2}[\,\ \p{L}\d\-]*\)'
    match_bad_year = r'[\S\s]*\((18|19|20)\d{2}\/(18|19|20)\d{2}\)'

    #we don't just have years, we also have sentences such as these instead of dates
    match_press = r'[\S\s]*\((i|I)n (P|p)ress|manuscript under review\)'
    match_forth = r'[\S\s]*\((f|F)orthcoming\)'
    match_accepted = r'[\S\s]*\((a|A)ccepted\)'
    match_submitted = r'[\S\s]*\((s|S)ubmitted\)'
    match_underreview = r'[\S\s]*\((u|U)nder (R|r)eview\)'

    #sentence = sentence.lower()
    if reg.match(regex, sentence):
        s = reg.search(regex, sentence).group(0)
        if len(s) > 9:
            return s
    elif re.match(match_bad_year, sentence):
        return re.search(match_bad_year, sentence).group(0)
    elif re.match(match_press, sentence):
        return re.search(match_press, sentence).group(0)
    elif re.match(match_forth, sentence):
        return re.search(match_forth, sentence).group(0)
    elif re.match(match_accepted, sentence):
        return re.search(match_accepted, sentence).group(0)
    elif re.match(match_submitted, sentence):
        return re.search(match_submitted, sentence).group(0)
    elif re.match(match_underreview, sentence):
        return re.search(match_underreview, sentence).group(0)

    return np.nan


def author_title(x):
    """Gets author and tite part of reference string"""
    ref = x
    authors = get_authors_month(x)  
    if isinstance(authors, float):
        return None
    
    search = len(authors)+1
    #In Looi is a comming occurence hence it is inlcuded here
    end = re.search('\.|\?|In Looi', ref[search:])
    if end:
        end = end.start()
    else:
        end = 0
    return ref[: (search+end)]

In [21]:
cleaning['identifier'] = cleaning[cleaning.citation.notna()].citation.map(lambda x: author_title(x))

## Unifying the names:


We see that often people spell their name differently across years, hence we try to unify the naming:

In [22]:
cleaning.long_name.sort_values().drop_duplicates().tail()

1349            van den Ende, Joan
2480    van der Schaaf, Marieke F.
3498             von Davier, Alina
3707          von Davier, Alina A.
804                   Öztok, Murat
Name: long_name, dtype: object

get the unique names and then use intersection to determine if one names is equal to another (we consider it equal if it contains at least two names that are equal)

In [23]:
names = cleaning.long_name.unique().tolist()

We "normalize" the names, that is convert them all into the same type of unicode formating to get even more overlap, otherwise, letters that look the same may not be matched by string comparison, as their unicode is different.

We visualy check that this way works by printing out matched. We can see that quite a lot of people have alternate versions of name spelling.

We will hoever keep some version of these alternate spellings to have a database of person names which will later be used when we extract affiliations from the papers themselves. Check the parsing universities notebook for more information. (Implemented in the scripts)

Reason why we normalize the string:

In [24]:
print(repr(u'ä'), ord(u'ä'))

for c in u"ä":
    print(repr(c), ord(c))

'ä' 228
'a' 97
'̈' 776


In [25]:
unicodedata.normalize('NFC', u'ä')

'ä'

In [26]:
d= {}
for i, m in enumerate(names):
    for j, n in enumerate(names):
        if i < j:
            y = set([i.lower() for i in reg.split(' |\,', unicodedata.normalize('NFC', m)) if len(reg.sub('\.', '', i)) > 2])
            name = set([i.lower() for i in reg.split(' |\,', unicodedata.normalize('NFC', n)) if len(reg.sub('\.', '', i)) > 2])
            if len(name.intersection(y)) > 1 and n!= m and not ('Lee' in n or 'Lee' in m):
                d[n]= m
                print(n, '|', m)


Wallon, Robert C. | Wallon, Robert C
Scipio, Déana Aeolani | Scipio, Déana
Asterhan, Christa S. C. | Asterhan, Christa
Suárez, Enrique | Suárez, Enrique
Michaelis, Joseph E. | Michaelis, Joseph E
Tomar, Gaurav | Tomar, Gaurav Singh
Chan, Carol K. K. | Chan, Carol
Chan, Carol K.K. | Chan, Carol
Sommer, Stephen | sommer, stephen
Schwarz, Baruch B. | Schwarz, Baruch
Pier, Elizabeth | Pier, Elizabeth L.
Searle, Kristin A. | Searle, Kristin A
Flood, Virginia J. | Flood, Virginia J
Schwendimann, Beat | Schwendimann, Beat A.
Bemis, Carrie Allen | Allen, Carrie D.
Gutiérrez, José | Gutiérrez, José Francisco
Derry, Sharon | Derry, Sharon J.
Wise, Alyssa | Wise, Alyssa Friend
Sandoval, William | Sandoval, William A.
McDonald, Scott P. | McDonald, Scott
Margulieux, Lauren E | Margulieux, Lauren E.
Penuel, William | Penuel, William R.
Lam, Rachel J. | Lam, Rachel
Wendell, Kristen | Wendell, Kristen B.
Clegg, Tamara L. | Clegg, Tamara
McBride, Elizabeth A. | McBride, Elizabeth
Olsen, Jennifer K. 

We check the number of current names so we can see how the unification of spelling reduces the number of unique names.

In [27]:
len(cleaning.long_name.unique())

2594

We build a mapping using the above matched, and we can now us it to unify the naming:

In [28]:
cleaning.loc[cleaning['long_name'].isin(d.keys()), 'long_name'] = cleaning.long_name.map(d)

In [29]:
len(cleaning.long_name.unique())

2451

We can se that we reduce the number of distinct names quite a bit!

In [30]:
import networkx as nx

In [31]:
build_graph = cleaning[['long_name', 'shortend_names', 'file']]
build_graph = pd.merge(build_graph, build_graph, on='file')
build_graph = build_graph[build_graph.long_name_x != build_graph.long_name_y]

In [32]:
build_graph.head()

,long_name_x,shortend_names_x,file,long_name_y,shortend_names_y
2,"Krishnan, Gokul",Krishnan G.,2014_1547-1548,"Sengupta, Pratim",Sengupta P.
3,"Sengupta, Pratim",Sengupta P.,2014_1547-1548,"Krishnan, Gokul",Krishnan G.
6,"Chen, Ying-Chih",Chen Y.,2014_641-648,"Hand, Brian",Hand B.
7,"Hand, Brian",Hand B.,2014_641-648,"Chen, Ying-Chih",Chen Y.
10,"Uttamchandani, Suraj",Uttamchandani S.,2018_367,"Peppler, Kylie A",Peppler K. A.


In [33]:
G = nx.from_pandas_edgelist(build_graph, source='long_name_x', target='long_name_y')

In [34]:
import difflib

In [35]:
threshold = 0.8
d = dict()
for name in build_graph.long_name_x.unique():
    members = sorted(list(G.neighbors(name)))
    for i, member in enumerate(members):
            for j, member_2 in enumerate(members):
                if i < j:
                    difference = difflib.SequenceMatcher(None, member, member_2).ratio()
                    if difference > threshold:
                        print(member, '|' ,member_2)
                        d[member] = member_2

Quintana, Rebecca | Quintana, Rebecca M
Silva, Brenda A. López | Silva, Brenda Lopez
Greenhow, Chrisine | Greenhow, Christine
Dascalu, Mihai | Dascălu, Mihai
Andrade, Alejandro | Andrade-Lotero, Alejandro
Rose, Carolyn | Rosé, Carolyn
Azevedo, Flavio S. | Azevedo, Flávio S.
*, Meixi | Ng, Meixi
Polman, Joseph L | Polman, Joseph L.
Hoadley, Christopher | Hovey, Christopher M
Greenhow, Chrisine | Greenhow, Christine
Greenhow, Chrisine | Greenhow, Christine
Greenhow, Chrisine | Greenhow, Christine
Hoadley, Christopher | Hovey, Christopher M
Barber-Lester, Kelly J. | Barber-Lester, Kelly Johnson
*, Meixi | Ng, Meixi
Rose, Carolyn | Rosé, Carolyn
Keifert, Danielle | Keifert, Danielle Teodora
Eagan, Brendan | Eagan, Brendan R.
Eagan, Brendan | Eagan, Brendan R.
Krämer, Nicole | Krämer, Nicole C.
Severance, Sam | Severance, Samuel
C.Linn, Marcia | Linn, Marcia
C.Linn, Marcia | Linn, Marcia
M.Vitale, Jonathan | Vitale, Jonathan
Madhok, Jacqueline | Madhok, Jacquie
C.Linn, Marcia | Linn, Marc

In [36]:
d

{'*, Meixi': 'Ng, Meixi',
 'Andrade, Alejandro': 'Andrade-Lotero, Alejandro',
 'Azevedo, Flavio S.': 'Azevedo, Flávio S.',
 'Barber-Lester, Kelly J.': 'Barber-Lester, Kelly Johnson',
 'C.Linn, Marcia': 'Linn, Marcia',
 'Cuendet, Sebastien': 'Cuendet, Sébastien',
 "D'Angelo, Sarah": 'D’Angelo, Sarah',
 'Dascalu, Mihai': 'Dascălu, Mihai',
 'Eagan, Brendan': 'Eagan, Brendan R.',
 'Gomez, Kimberley': 'Gomez, Kimberly',
 'Greenhow, Chrisine': 'Greenhow, Christine',
 'Hoadley, Christopher': 'Hovey, Christopher M',
 'Keifert, Danielle': 'Keifert, Danielle Teodora',
 'Koehler, Jessica': 'Koehler-Yom, Jessica',
 'Krajcik, Joe': 'Krajcik, Joseph',
 'Krämer, Nicole': 'Krämer, Nicole C.',
 'Lin, Chiu Pin': 'Lin, Chiu-Pin',
 'Lyons, Leila': 'Lyons, Leilah',
 'M.Vitale, Jonathan': 'Vitale, Jonathan',
 'Madhok, Jacqueline': 'Madhok, Jacquie',
 'Martin, Lee': 'Martin, Lee M',
 'Polman, Joseph L': 'Polman, Joseph L.',
 'Quintana, Rebecca': 'Quintana, Rebecca M',
 'Rose, Carolyn': 'Rosé, Carolyn',
 'S

In [37]:
cleaning.loc[cleaning['long_name'].isin(d.keys()), 'long_name'] = cleaning.long_name.map(d)

In [38]:
cleaning.long_name.drop_duplicates().shape

(2420,)

Now that the dataset is clean we can explore it, check the analysis section for more information.